file: cone-bucket-train.ipynb  
site: github.com/IYATT-yx/learning  
describe: 锥桶检测迁移学习训练  

该笔记本可直接在 Google Colaboratory 上运行，运行前请将本笔记本文件及 cone-bucket.tar.gz （锥桶图片和标注数据）和 generate_tfrecord.py (xml 标注转 tf 数据）拷贝到 Google Drive 上，设置 GPU 加速然后运行本笔记本。  

Copyright (C) 2021 IYATT-yx (Zhao Hongfei, 赵洪飞)，2514374431@qq.com
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU Affero General Public License as published
by the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU Affero General Public License for more details.
You should have received a copy of the GNU Affero General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

## 环境准备

In [1]:
!pip3 install opencv-python==4.5.3.56 tensorflow==2.6.0 matplotlib==3.4.3 protobuf==3.18.0 google==3.0.0

     |████████████████████████████████| 49.9 MB 18 kB/s 
     |████████████████████████████████| 10.3 MB 24.7 MB/s 
     |████████████████████████████████| 1.1 MB 28.8 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
Reason for being yanked: This version claims to support Python 2 but does not
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: google
    Found existing installation: google 2.0.3
    Uninstalling google-2.0.3:
      Successfully uninstalled google-2.0.3
ERROR: pip's depende

## 资源路径设置

In [2]:
# 挂载谷歌云盘
from google.colab import drive
drive.mount('/content/temp', force_remount=True)

Mounted at /content/temp


In [3]:
# 将资源文件拷贝到 Colab
!cp temp/MyDrive/锥桶识别/cone-bucket.tar.gz .
!cp temp/MyDrive/锥桶识别/generate_tfrecord.py .
!ls

cone-bucket.tar.gz  generate_tfrecord.py  sample_data  temp


In [4]:
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [5]:
paths = {
    'WORKSPACE_PATH': 'work-temp/workspace',
    'SCRIPTS_PATH': 'work-temp/scripts',
    'APIMODEL_PATH': 'work-temp/models',
    'ANNOTATION_PATH': 'work-temp/workspace/annotations',
    'IMAGE_PATH': 'work-temp/workspace/images',
    'MODEL_PATH': 'work-temp/workspace/models',
    'PRETRAINED_MODEL_PATH': 'work-temp/workspace/pretrained_models',
    'CHECKPOINT_PATH': 'work-temp/workspace/models',  
    'PROTOC_PATH': 'work-temp/protoc'
 }

In [6]:
files = {
    'PIPELINE_CONFIG': 'work-temp/workspace/models/' + 'pipeline.config',
    'TF_RECORD_SCRIPT': paths['SCRIPTS_PATH'] + '/' + TF_RECORD_SCRIPT_NAME,
    'LABELMAP': paths['ANNOTATION_PATH'] + '/' + LABEL_MAP_NAME
}

In [7]:
# 创建工作路径
import os
for path in paths.values():
    if not os.path.exists(path):
        !mkdir -p {path}
!apt install tree
!tree work-temp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (104 kB/s)
Selecting previously unselected package tree.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
work-temp
├── models
├── protoc
├── scripts
└── workspace
    ├── annotations
    ├── images
    ├── models
    └── pretrained_models

8 directories, 0 files


## 获取预训练模型及安装 ob

In [8]:
!git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']} --depth=1

Cloning into 'work-temp/models'...
remote: Enumerating objects: 3023, done.
remote: Counting objects: 100% (3023/3023), done.
remote: Compressing objects: 100% (2552/2552), done.
remote: Total 3023 (delta 757), reused 1307 (delta 432), pack-reused 0
Receiving objects: 100% (3023/3023), 33.23 MiB | 27.23 MiB/s, done.
Resolving deltas: 100% (757/757), done.


In [9]:
!cd work-temp/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && pip3 install . 

Processing /content/work-temp/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 6.8 MB/s 
     |████████████████████████████████| 352 kB 40.3 MB/s 
     |████████████████████████████████| 1.8 MB 37.4 MB/s 
     |████████████████████████████████| 99 kB 9.2 MB/s 
     |████████████████████████████████| 1.2 MB 42.6 MB/s 
     |████████████████████████████████| 596 kB 18.9 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 90 kB 9.3 MB/s 
     |████████████████████████████████| 213 kB 40.8 MB/s 
     |████████████████████████████████| 4.4

In [10]:
# 下载预训练模型
!wget {PRETRAINED_MODEL_URL}
!mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2021-10-26 01:29:47--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 209.85.146.128, 2607:f8b0:4001:c1f::80
Connecting to download.tensorflow.org (download.tensorflow.org)|209.85.146.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M   117MB/s    in 0.2s    

2021-10-26 01:29:48 (117 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8

## 创建标签图文件

In [11]:
labels = [{'name':'cone-bucket', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## 转换数据

In [12]:
# OPTIONAL IF RUNNING ON COLAB
ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
if os.path.exists(ARCHIVE_FILES):
  !tar -zxvf {ARCHIVE_FILES}

In [13]:
!tar xvf cone-bucket.tar.gz

cone-bucket/
cone-bucket/train/
cone-bucket/train/17.jpeg
cone-bucket/train/11.jpeg
cone-bucket/train/21.xml
cone-bucket/train/4.xml
cone-bucket/train/34.xml
cone-bucket/train/21.jpeg
cone-bucket/train/5.jpeg
cone-bucket/train/36.jpeg
cone-bucket/train/22.xml
cone-bucket/train/35.xml
cone-bucket/train/7.jpeg
cone-bucket/train/25.xml
cone-bucket/train/13.jpeg
cone-bucket/train/7.xml
cone-bucket/train/23.xml
cone-bucket/train/19.xml
cone-bucket/train/33.xml
cone-bucket/train/26.xml
cone-bucket/train/4.jpeg
cone-bucket/train/32.xml
cone-bucket/train/14.jpeg
cone-bucket/train/2.jpeg
cone-bucket/train/30.jpeg
cone-bucket/train/8.jpeg
cone-bucket/train/20.xml
cone-bucket/train/12.xml
cone-bucket/train/15.jpeg
cone-bucket/train/22.jpeg
cone-bucket/train/24.jpeg
cone-bucket/train/9.xml
cone-bucket/train/28.xml
cone-bucket/train/31.jpeg
cone-bucket/train/16.jpeg
cone-bucket/train/14.xml
cone-bucket/train/13.xml
cone-bucket/train/2.xml
cone-bucket/train/27.jpeg
cone-bucket/train/10.jpeg
cone-buc

In [14]:
!cp generate_tfrecord.py {paths['SCRIPTS_PATH']}
!cp -r cone-bucket/* {paths['IMAGE_PATH']}

In [15]:
!python3 {files['TF_RECORD_SCRIPT']} -x {paths['IMAGE_PATH'] + '/train/'} -l {files['LABELMAP']} -o {paths['ANNOTATION_PATH'] + '/train.record'} 
!python3 {files['TF_RECORD_SCRIPT']} -x {paths['IMAGE_PATH'] + '/test/'} -l {files['LABELMAP']} -o {paths['ANNOTATION_PATH'] + '/test.record'}

Successfully created the TFRecord file: work-temp/workspace/annotations/train.record
Successfully created the TFRecord file: work-temp/workspace/annotations/test.record


## 复制模型配置到训练文件夹

In [16]:
!cp {paths['PRETRAINED_MODEL_PATH'] + '/' + PRETRAINED_MODEL_NAME + '/pipeline.config'} {paths['CHECKPOINT_PATH']}

## 迁移学习配置

In [17]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [18]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [19]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = paths['PRETRAINED_MODEL_PATH'] + '/' + PRETRAINED_MODEL_NAME + '/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [paths['ANNOTATION_PATH'] + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [paths['ANNOTATION_PATH'] + '/test.record']

In [20]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

## 训练模型

In [21]:
TRAINING_SCRIPT = paths['APIMODEL_PATH'] + '/research/object_detection/model_main_tf2.py'
cmd = "python3 {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])
print(cmd)
!{cmd}

python3 work-temp/models/research/object_detection/model_main_tf2.py --model_dir=work-temp/workspace/models --pipeline_config_path=work-temp/workspace/models/pipeline.config --num_train_steps=2000
2021-10-26 01:30:01.815352: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-26 01:30:01.846616: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-26 01:30:01.847482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-26 01:30:01.849376: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value

## 载入训练模型

In [22]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [24]:
# 载入 pipeline config 和构建一个检测模型
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 测试检测效果

In [25]:
import cv2 
import numpy as np

In [26]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])
 
for i in range(1, 8):
    img = cv2.imread(paths['IMAGE_PATH'] + '/test/' + str(i) + '.jpeg')
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)
    cv2.imwrite(str(i) + '.jpeg', image_np_with_detections)  # 保存检测结果

In [30]:
from google.colab import files
for i in range(1, 8):
    files.download(str(i) + '.jpeg')  # 下载检测结果

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 下载模型

In [31]:
i = 1
while True:
    try:
        files.download('work-temp/workspace/models/ckpt-' + str(i) + '.data-00000-of-00001')
        files.download('work-temp/workspace/models/ckpt-' + str(i) + '.index')
    except:
        break
    i = i + 1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>